In [4]:
!pip3 install tensorflow
!pip3 install keras
!pip3 install h5py
!pip3 install sklearn
!pip3 install scikit-image

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 13.4 MB 22.6 MB/s eta 0:00:01
     |██████████████

In [12]:
!pip3 uninstall -y numpy

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4


In [13]:
!pip3 install numpy<2
!pip3 install --upgrade scikit-learn
!pip3 install --upgrade tensorflow


zsh:1: no such file or directory: 2
Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-2.0.2-cp39-cp39-macosx_14_0_arm64.whl.metadata (60 kB)
Using cached numpy-2.0.2-cp39-cp39-macosx_14_0_arm64.whl (5.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.0 requires numpy<2.0.0,>=1.23.5; python_version <= "3.11", but you have numpy 2.0.2 which is incompatible.
Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-1.26.4-cp39-cp39-macosx_11_0_arm64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp39-cp39-macosx_11_0_arm64.whl (14.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2


In [15]:
!pip3 uninstall -y tensorflow

Found existing installation: tensorflow 2.17.0
Uninstalling tensorflow-2.17.0:
  Successfully uninstalled tensorflow-2.17.0


In [16]:
!pip3 install tensorflow 

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.1/236.1 MB 27.4 MB/s eta 0:00:00m eta 0:00:010:01:02


In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import h5py
from sklearn.model_selection import train_test_split
from skimage.transform import resize
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input

/Users/edgardocrovetto/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:
!pip3 install tqdm
from tqdm import tqdm  # For progress tracking

Defaulting to user installation because normal site-packages is not writeable


In [10]:
# Cargar metadatos
df = pd.read_csv('/Users/edgardocrovetto/Library/CloudStorage/OneDrive-Personal/Documentos/M1000IA/datasets/metadatos_actualizados_limpios_normalizados.csv')

df['image_name'] = df['image_id']


In [11]:
df.head()

,lesion_id,image_id,dx,age,sex,localization,classification,dataset,image_name
0,ham_0003577,isic_0024307.jpg,nv,50.0,male,lower extremity,lesión benigna,original,isic_0024307.jpg
1,ham_0000484,isic_0024309.jpg,nv,40.0,male,trunk,lesión benigna,original,isic_0024309.jpg
2,ham_0003350,isic_0024310.jpg,mel,60.0,male,chest,lesión maligna,original,isic_0024310.jpg
3,ham_0001359,isic_0024312.jpg,bkl,75.0,male,lower extremity,lesión benigna,original,isic_0024312.jpg
4,ham_0002869,isic_0024313.jpg,mel,50.0,female,back,lesión maligna,original,isic_0024313.jpg


In [17]:
# Function to load and preprocess image as a TensorFlow tensor
def load_image(image_name, image_dir, target_size=(112, 112)):
    image_path = os.path.join(image_dir, image_name)
    try:
        img = tf.io.read_file(image_path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, target_size)  # Resize to target size
        img = img / 255.0  # Normalize pixel values to [0, 1]
        return img.numpy()  # Convert to numpy array for HDF5 storage
    except Exception as e:
        print(f"Error loading image {image_name}: {e}")
        return None

# Directory where images are stored
image_dir = '/Users/edgardocrovetto/Library/CloudStorage/OneDrive-Personal/Documentos/M1000IA/datasets/merged_images'

# Apply load_image function to each image, while showing a progress bar
imagenes_transformadas = []
for image_name in tqdm(df['image_name'], desc="Processing images"):
    img = load_image(image_name, image_dir)
    if img is not None:
        imagenes_transformadas.append(img)

# Convert the list of images to a numpy array (excluding any failed loads)
imagenes_array = np.stack(imagenes_transformadas)

# Convert labels to one-hot encoding (assuming 'classification' column is categorical)
etiquetas = pd.get_dummies(df['classification']).values

# Print generated class names from one-hot encoding
nombres_clases = list(pd.get_dummies(df['classification']).columns)
print("Generated class labels:", nombres_clases)

# Save the images and labels to an HDF5 file
with h5py.File('dataset_y_clasificacion.h5', 'w') as hdf:
    hdf.create_dataset('images', data=imagenes_array)
    hdf.create_dataset('classification', data=etiquetas)

print("Dataset creation complete.")


Processing images: 100%|██████████| 6021/6021 [00:32<00:00, 183.05it/s]


Generated class labels: ['lesión benigna', 'lesión maligna']
Dataset creation complete.


In [15]:
# Nombre actual del archivo
archivo_viejo = 'datasetMerged2Etiquetas.h5'

# Nuevo nombre del archivo
archivo_nuevo = 'dataset_y_clasificacion.h5'

# Renombrar el archivo
os.rename(archivo_viejo, archivo_nuevo)

print(f"El archivo ha sido renombrado a: {archivo_nuevo}")

El archivo ha sido renombrado a: dataset_y_clasificacion.h5


In [18]:
# Lista para almacenar las imágenes transformadas
imagenes_transformadas_metadatos = []
# Almacenar los índices de las filas para las imágenes que no pudieron cargarse
failed_indices_metadatos = []

# Aplicar la función load_image a cada imagen y vincular con los metadatos
for idx, image_name in tqdm(enumerate(df['image_name']), desc="Processing images", total=len(df)):
    img = load_image(image_name, image_dir)
    if img is not None:
        imagenes_transformadas_metadatos.append(img)
    else:
        failed_indices_metadatos.append(idx)

# Eliminar las filas correspondientes a imágenes fallidas
df_cleaned = df.drop(failed_indices_metadatos)

# Convertir la lista de imágenes en un array de numpy
if imagenes_transformadas_metadatos:
    imagenes_array = np.stack(imagenes_transformadas_metadatos)
else:
    raise ValueError("No se cargaron imágenes correctamente.")

# Guardar las imágenes y los metadatos en un archivo HDF5
with h5py.File('dataset_con_metadatos.h5', 'w') as hdf:
    # Guardar las imágenes
    hdf.create_dataset('images', data=imagenes_array)
    
    # Guardar las columnas de metadatos vinculadas
    hdf.create_dataset('age', data=df_cleaned['age'].values)
    hdf.create_dataset('sex', data=df_cleaned['sex'].values.astype('S'))  # Convertir cadenas a formato binario
    hdf.create_dataset('localization', data=pd.get_dummies(df_cleaned['localization']).values)  # Convertir cadenas a formato binario
    hdf.create_dataset('classification', data=pd.get_dummies(df_cleaned['classification']).values)  # One-hot encoding de 'classification'

print("Archivo HDF5 con metadatos e imágenes creado exitosamente.")

Processing images: 100%|██████████| 6021/6021 [00:32<00:00, 184.68it/s]


Archivo HDF5 con metadatos e imágenes creado exitosamente.
